In [1]:
import pandas as pd
import snscrape.modules.twitter as sntwitter
import itertools
import time
from datetime import date, timedelta
import datetime
from tqdm import tqdm
from interruptingcow import timeout
from interruptingcow import timeout
import re
import string
punctuation_string = string.punctuation
tqdm.pandas()
import gc

Top 100 products from dataset
plus news channels
plus Martin Lewis
What else?

In [2]:
df = pd.read_csv('Datasetjson.csv', low_memory=False)
df.head()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1450029 entries, 0 to 1450028
Data columns (total 12 columns):
 #   Column          Non-Null Count    Dtype  
---  ------          --------------    -----  
 0   overall         1450029 non-null  float64
 1   vote            243082 non-null   object 
 2   verified        1450029 non-null  bool   
 3   reviewTime      1450029 non-null  object 
 4   reviewerID      1450029 non-null  object 
 5   asin            1450029 non-null  object 
 6   reviewerName    1449889 non-null  object 
 7   reviewText      1449370 non-null  object 
 8   summary         1449738 non-null  object 
 9   unixReviewTime  1450029 non-null  int64  
 10  style           839997 non-null   object 
 11  image           11566 non-null    object 
dtypes: bool(1), float64(1), int64(1), object(9)
memory usage: 123.1+ MB


In [3]:
df_meta= pd.read_csv('meta.csv', low_memory=False)
df_meta.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14751104 entries, 0 to 14751103
Data columns (total 3 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   brand     object
 1   asin      object
 2   category  object
dtypes: object(3)
memory usage: 337.6+ MB


In [4]:
df['review'] = df['reviewText']+' '+df['summary']
df['review'].head()

0    I bought two of these 8.5 fl oz hand cream, an...
1    Believe me, over the years I have tried many, ...
2                         Great hand lotion Five Stars
3    This is the best for the severely dry skin on ...
4    The best non- oily hand cream ever. It heals o...
Name: review, dtype: object

In [5]:
df.groupby('asin').count()
df[df['asin']=='0209688726']

,overall,vote,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,style,image,review
50001,4.0,3,True,"08 12, 2015",A3ELV1VUCZZGL4,0209688726,AnaG,it's fine. I just would like the stickers to b...,Serve its purpose,1439337600,{'Color:': ' CA'},['https://images-na.ssl-images-amazon.com/imag...,it's fine. I just would like the stickers to b...
50002,2.0,NaN,True,"08 12, 2015",A2FD3FZ2WV6KHQ,0209688726,Obi John Kenobi,took me three returns to get one that didn't w...,and it's still not super stable but whatever,1439337600,{'Color:': ' T'},NaN,took me three returns to get one that didn't w...
50003,2.0,12,True,"08 11, 2015",A39NRKABAVSVSE,0209688726,MuddyRiverGirl,While the product is fine the description and ...,Beware! Description is wrong.,1439251200,{'Color:': ' AC'},['https://images-na.ssl-images-amazon.com/imag...,While the product is fine the description and ...
50004,5.0,NaN,True,"07 29, 2015",AQSBL5KODHSGW,0209688726,Nate S,It's beautiful and blends right in with my woo...,"Awesome desk stand, simple and elegant",1438128000,{'Color:': ' T'},NaN,It's beautiful and blends right in with my woo...
50005,5.0,NaN,False,"07 11, 2015",AU45OQPK2J5JF,0209688726,Kort,I love this stand! I had been looking around f...,I love this stand,1436572800,{'Color:': ' T'},NaN,I love this stand! I had been looking around f...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
51242,5.0,NaN,False,"09 10, 2015",A3EGBGSVUD7PVH,0209688726,Momma gamma,I purchased two different ultrasonic diffusers...,excellent first diffuser,1441843200,{'Color:': ' J'},NaN,I purchased two different ultrasonic diffusers...
51243,5.0,NaN,True,"09 1, 2015",A8UYC4W0H6MP0,0209688726,Faaiz Khan,Everyone notices this on my desk! I am even co...,love it. simple and sleek,1441065600,{'Color:': ' T'},NaN,Everyone notices this on my desk! I am even co...
51244,3.0,NaN,True,"08 25, 2015",A3NNZTYM2TVNN8,0209688726,Kindle Customer,Holds 1-3 keys great but hard to get one set o...,Holds 1-3 keys great but hard to get one set o...,1440460800,{'Color:': ' CA'},NaN,Holds 1-3 keys great but hard to get one set o...
51245,5.0,4,True,"08 22, 2015",A3NMHKHMUNFB33,0209688726,Stacy King,I love how compact and easy to move rooms this...,Petite & Powerful,1440201600,{'Color:': ' I'},NaN,I love how compact and easy to move rooms this...


In [6]:
df_meta = df_meta.drop_duplicates(subset=['asin', 'brand', 'category'])
df_meta

,brand,asin,category
0,BubuBibi,0000032042,"['Sports & Outdoors', 'Sports & Fitness', 'Oth..."
1,BubuBibi,0000032069,"['Sports & Outdoors', 'Sports & Fitness', 'Oth..."
2,Unknown,0000031860,"['Sports & Outdoors', 'Sports & Fitness', 'Oth..."
3,Tutu,0000031852,"['Sports & Outdoors', 'Sports & Fitness', 'Oth..."
4,Bububibi,0000031895,"['Sports & Outdoors', 'Sports & Fitness', 'Oth..."
...,...,...,...
14751099,NaN,B01GMFJ8JU,"['Gift Cards', 'Gift Cards']"
14751100,QuickGifts,B01GOQIKRI,"['Gift Cards', 'Gift Cards']"
14751101,QuickGifts,B01GOQHGL4,"['Gift Cards', 'Gift Cards']"
14751102,Go Play Golf by Fairway Rewards,B01GP1W4LA,"['Gift Cards', 'Gift Cards']"


In [7]:
df_meta.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14064249 entries, 0 to 14751103
Data columns (total 3 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   brand     object
 1   asin      object
 2   category  object
dtypes: object(3)
memory usage: 429.2+ MB


In [8]:
df_merge = df.merge(df_meta, on='asin')

In [9]:
df_merge

,overall,vote,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,style,image,review,brand,category
0,2.0,3,True,"06 15, 2010",A1Q6MUU0B2ZDQG,B00004U9V2,D. Poston,"I bought two of these 8.5 fl oz hand cream, an...",dispensers don't work,1276560000,NaN,NaN,"I bought two of these 8.5 fl oz hand cream, an...",NaN,[]
1,5.0,14,True,"01 7, 2010",A3HO2SQDCZIE9S,B00004U9V2,chandra,"Believe me, over the years I have tried many, ...",Best hand cream ever.,1262822400,NaN,NaN,"Believe me, over the years I have tried many, ...",NaN,[]
2,5.0,NaN,True,"04 18, 2018",A2EM03F99X3RJZ,B00004U9V2,Maureen G,Great hand lotion,Five Stars,1524009600,{'Size:': ' 3.5 oz.'},NaN,Great hand lotion Five Stars,NaN,[]
3,5.0,NaN,True,"04 18, 2018",A3Z74TDRGD0HU,B00004U9V2,Terry K,This is the best for the severely dry skin on ...,Five Stars,1524009600,{'Size:': ' 3.5 oz.'},NaN,This is the best for the severely dry skin on ...,NaN,[]
4,5.0,NaN,True,"04 17, 2018",A2UXFNW9RTL4VM,B00004U9V2,Patricia Wood,The best non- oily hand cream ever. It heals o...,I always have a backup ready.,1523923200,{'Size:': ' 3.5 oz.'},NaN,The best non- oily hand cream ever. It heals o...,NaN,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1331153,5.0,NaN,True,"01 22, 2014",A3UH3CTPNSKQW2,B001D1Z7UI,Amazon Customer,This cookbook is awsome! I had gastric bypass ...,Finally a way to start the weight loss agian w...,1390348800,NaN,NaN,This cookbook is awsome! I had gastric bypass ...,Visit Amazon's Patt Levine Page,"['Kindle Store', 'Kindle eBooks', 'Health, Fit..."
1331154,4.0,NaN,True,"01 21, 2014",A304BUBREQKRJY,B001D1Z7UI,Tricia Smith,I bought this before weight loss surgery it he...,helpful,1390262400,NaN,NaN,I bought this before weight loss surgery it he...,Visit Amazon's Patt Levine Page,"['Kindle Store', 'Kindle eBooks', 'Health, Fit..."
1331155,5.0,NaN,True,"12 30, 2013",A3EIQ5QQKYMTJQ,B001D1Z7UI,Book_thyme,I got this for my mom who had traditional bari...,not just for bariatric surgery patients,1388361600,NaN,NaN,I got this for my mom who had traditional bari...,Visit Amazon's Patt Levine Page,"['Kindle Store', 'Kindle eBooks', 'Health, Fit..."
1331156,5.0,NaN,True,"12 27, 2013",A2B7ANEHU0OYKJ,B001D1Z7UI,Leslie J Riney,I thought it was slow in arrival but it was as...,good book,1388102400,NaN,NaN,I thought it was slow in arrival but it was as...,Visit Amazon's Patt Levine Page,"['Kindle Store', 'Kindle eBooks', 'Health, Fit..."


In [10]:
df_merge[df_merge.brand.notnull()].groupby('brand').count().sort_values(by='reviewText', ascending=False)

,overall,vote,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,style,image,review,category
brand,,,,,,,,,,,,,,
nobrand,23277,2171,23277,23277,23277,23277,23276,23256,23267,23277,18672,423,23251,23277
OXO,21783,2512,21783,21783,21783,21783,21778,21782,21780,21783,18167,106,21779,21783
Amazon,14825,727,14825,14825,14825,14825,14823,14815,14818,14825,11393,36,14808,14825
Microsoft,12030,5525,12030,12030,12030,12030,12029,12030,12029,12030,2591,16,12029,12030
Motorola,11079,3113,11079,11079,11079,11079,11079,11078,11079,11079,3390,31,11078,11079
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Fox Chapel Publishing Co Inc,1,1,1,1,1,1,1,1,1,1,0,0,1,1
Ajika,1,1,1,1,1,1,1,1,1,1,0,0,1,1
Debbie Kerner Rettino,1,1,1,1,1,1,1,1,1,1,1,0,1,1


In [11]:
df_merge.dropna().reset_index(drop=True)

,overall,vote,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,style,image,review,brand,category
0,1.0,4,True,"10 1, 2016",A3MMO1UVNM7D0H,B00002243Z,T. Andrew,Very poorly made in China. Look at the pictur...,Garbage. Look elsewhere,1475280000,{'Size:': ' 25-Feet'},['https://images-na.ssl-images-amazon.com/imag...,Very poorly made in China. Look at the pictur...,Coleman Cable,"['Automotive', 'Replacement Parts', 'Batteries..."
1,3.0,131,True,"03 28, 2010",AWKYTPVNNZELL,B00002243Z,T. Brown,For the price and in the right situations thes...,"Good Jumper Cables for the money, But.....",1269734400,{'Size:': ' 20-Feet'},['https://images-na.ssl-images-amazon.com/imag...,For the price and in the right situations thes...,Coleman Cable,"['Automotive', 'Replacement Parts', 'Batteries..."
2,3.0,5,True,"09 17, 2016",A242UHNRH3GNQK,B00004T81V,welder314,This tool is a usable option if you do occasio...,"Nice concept, poor execution",1474070400,{'Style:': ' Plier Type'},['https://images-na.ssl-images-amazon.com/imag...,This tool is a usable option if you do occasio...,General Tools,"['Automotive', 'Tools & Equipment', 'Brake Rep..."
3,3.0,13,True,"12 5, 2015",AO53T9NBLHN1Q,B00004T81V,Juan S.,It really didn't help that no one might of tok...,Not really what I expected,1449273600,{'Style:': ' Plier Type'},['https://images-na.ssl-images-amazon.com/imag...,It really didn't help that no one might of tok...,General Tools,"['Automotive', 'Tools & Equipment', 'Brake Rep..."
4,2.0,8,True,"05 14, 2016",A5VCQ0V8WDDCQ,B000056OQL,big_art,I was excited to try this on my new Porter Cab...,Foam Pad Separated From Disc - First Use,1463184000,{'Package Quantity:': ' 1'},['https://images-na.ssl-images-amazon.com/imag...,I was excited to try this on my new Porter Cab...,PORTER-CABLE,"['Automotive', 'Tools & Equipment', 'Body Repa..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3719,5.0,2,False,"12 18, 2016",A23VCN0WFODLT,B000ZN24OW,Andy,"In my opinion, this is one of the best books t...",This book really woke me up to my true power!,1482019200,{'Format:': ' Paperback'},['https://images-na.ssl-images-amazon.com/imag...,"In my opinion, this is one of the best books t...",Visit Amazon's Wallace D. Wattles Page,"['Kindle Store', 'Kindle eBooks', 'Religion & ..."
3720,5.0,2,False,"08 5, 2017",A3IY97EOJU0ATJ,B0013TPY3A,Jon Meyer,I use to have a version of this book that look...,... to have a version of this book that looked...,1501891200,{'Format:': ' Hardcover'},['https://images-na.ssl-images-amazon.com/imag...,I use to have a version of this book that look...,Visit Amazon's A. A. Milne Page,"['Kindle Store', 'Kindle eBooks', ""Children's ..."
3721,5.0,20,True,"12 16, 2016",A569MQUDC5ELX,B0013TPY3A,Angie,Book came quickly despite the crazy weather. I...,Beautifully crafted book filled with memories.,1481846400,{'Format:': ' Hardcover'},['https://images-na.ssl-images-amazon.com/imag...,Book came quickly despite the crazy weather. I...,Visit Amazon's A. A. Milne Page,"['Kindle Store', 'Kindle eBooks', ""Children's ..."
3722,5.0,2,False,"08 5, 2017",A3IY97EOJU0ATJ,B0013TPY3A,Jon Meyer,I use to have a version of this book that look...,... to have a version of this book that looked...,1501891200,{'Format:': ' Hardcover'},['https://images-na.ssl-images-amazon.com/imag...,I use to have a version of this book that look...,Visit Amazon's A. A. Milne Page,"['Kindle Store', 'Kindle eBooks', ""Children's ..."


In [12]:
df_merge[df_merge['brand'].notnull()]

,overall,vote,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,style,image,review,brand,category
50001,5.0,NaN,True,"03 20, 2015",A3M20IGF19WMBD,7138258879,julie,The were good,Five Stars,1426809600,NaN,NaN,The were good Five Stars,Elite Mailers,"['Automotive', 'Exterior Accessories', 'Bumper..."
50002,3.0,NaN,True,"01 14, 2015",A2GGCRLQ75WJFE,7138258879,Subiechick,One sticker was incomplete and missing letters...,Incomplete,1421193600,NaN,NaN,One sticker was incomplete and missing letters...,Elite Mailers,"['Automotive', 'Exterior Accessories', 'Bumper..."
50003,5.0,NaN,True,"01 3, 2015",A2M7MSLDRV6FC6,7138258879,Cris Canoneo,Thank you,Five Stars,1420243200,NaN,NaN,Thank you Five Stars,Elite Mailers,"['Automotive', 'Exterior Accessories', 'Bumper..."
50004,5.0,NaN,False,"11 1, 2014",A3RKXT2JD6KZR0,7138258879,Mr. Sam,Great space filler that will impress a lot of ...,Great space filler that will impress a lot of ...,1414800000,NaN,NaN,Great space filler that will impress a lot of ...,Elite Mailers,"['Automotive', 'Exterior Accessories', 'Bumper..."
50005,5.0,NaN,False,"09 14, 2014",A2IJ5951R8OXLT,7138258879,Silveradolife04,Great product sticks well gone through several...,Fantastic,1410652800,NaN,['https://images-na.ssl-images-amazon.com/imag...,Great product sticks well gone through several...,Elite Mailers,"['Automotive', 'Exterior Accessories', 'Bumper..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1331153,5.0,NaN,True,"01 22, 2014",A3UH3CTPNSKQW2,B001D1Z7UI,Amazon Customer,This cookbook is awsome! I had gastric bypass ...,Finally a way to start the weight loss agian w...,1390348800,NaN,NaN,This cookbook is awsome! I had gastric bypass ...,Visit Amazon's Patt Levine Page,"['Kindle Store', 'Kindle eBooks', 'Health, Fit..."
1331154,4.0,NaN,True,"01 21, 2014",A304BUBREQKRJY,B001D1Z7UI,Tricia Smith,I bought this before weight loss surgery it he...,helpful,1390262400,NaN,NaN,I bought this before weight loss surgery it he...,Visit Amazon's Patt Levine Page,"['Kindle Store', 'Kindle eBooks', 'Health, Fit..."
1331155,5.0,NaN,True,"12 30, 2013",A3EIQ5QQKYMTJQ,B001D1Z7UI,Book_thyme,I got this for my mom who had traditional bari...,not just for bariatric surgery patients,1388361600,NaN,NaN,I got this for my mom who had traditional bari...,Visit Amazon's Patt Levine Page,"['Kindle Store', 'Kindle eBooks', 'Health, Fit..."
1331156,5.0,NaN,True,"12 27, 2013",A2B7ANEHU0OYKJ,B001D1Z7UI,Leslie J Riney,I thought it was slow in arrival but it was as...,good book,1388102400,NaN,NaN,I thought it was slow in arrival but it was as...,Visit Amazon's Patt Levine Page,"['Kindle Store', 'Kindle eBooks', 'Health, Fit..."


In [13]:
def cleaning_punctuations(text):
    translator = str.maketrans('', '', punctuation_string)
    return text.translate(translator)

#df_merge = df_merge.dropna().reset_index(drop=True)
df_merge = df_merge[df_merge['brand'].notnull()]
print('cleaning punctuations')
clean_punct = df_merge['brand'].progress_apply(lambda x: cleaning_punctuations(x))
df_merge = df_merge.assign(brand=clean_punct)

cleaning punctuations


100%|██████████| 1234594/1234594 [00:21<00:00, 58183.67it/s]


In [14]:
remove_visit = df_merge['brand'].progress_apply(lambda x: re.sub('Visit Amazons ','', x))
df_merge= df_merge.assign(brand=remove_visit)

100%|██████████| 1234594/1234594 [00:10<00:00, 122394.63it/s]


In [15]:
remove_amazonpage = df_merge['brand'].progress_apply(lambda x: re.sub(' Page','', x))

100%|██████████| 1234594/1234594 [00:09<00:00, 132696.11it/s]


In [16]:
df_merge= df_merge.assign(brand=remove_amazonpage)

In [17]:
df_merge.groupby('brand').count()

,overall,vote,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,style,image,review,category
brand,,,,,,,,,,,,,,
Gary Puckett the Union Gap,48,14,48,48,48,48,48,48,48,48,48,0,48,48
101 Strings Orchestra,56,12,56,56,56,56,56,56,56,56,56,0,56,56
1080,26,5,26,26,26,26,26,26,26,26,0,0,26,26
10cc,1,1,1,1,1,1,1,1,1,1,1,0,1,1
13Rainbow,9,2,9,9,9,9,9,9,9,9,0,0,9,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
xCers Corp,21,8,21,21,21,21,21,21,21,21,0,0,21,21
xheli,18,10,18,18,18,18,18,18,18,18,0,0,18,18
xxx,3,2,3,3,3,3,3,3,3,3,0,0,3,3


In [18]:
text = 'Visit Amazons Zondervan Page'
re.search('Visit Amazons [a-zA-Z]+ Page', text)

<re.Match object; span=(0, 28), match='Visit Amazons Zondervan Page'>

In [19]:
remove_by = df_merge['brand'].progress_apply(lambda x: re.sub('by\n    \n    ','' ,x))

100%|██████████| 1234594/1234594 [00:07<00:00, 164508.93it/s]


In [20]:
df_merge= df_merge.assign(brand=remove_by)
#df_merge.loc[18624]

In [21]:
df_products = df_merge.groupby('brand').count().sort_values('overall', ascending=False)

In [22]:
df_products.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5925 entries, nobrand to Shadowfax
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype
---  ------          --------------  -----
 0   overall         5925 non-null   int64
 1   vote            5925 non-null   int64
 2   verified        5925 non-null   int64
 3   reviewTime      5925 non-null   int64
 4   reviewerID      5925 non-null   int64
 5   asin            5925 non-null   int64
 6   reviewerName    5925 non-null   int64
 7   reviewText      5925 non-null   int64
 8   summary         5925 non-null   int64
 9   unixReviewTime  5925 non-null   int64
 10  style           5925 non-null   int64
 11  image           5925 non-null   int64
 12  review          5925 non-null   int64
 13  category        5925 non-null   int64
dtypes: int64(14)
memory usage: 694.3+ KB


In [23]:
df_products = df_products.rename_axis('brand').reset_index()


In [24]:
top_100 = df_products['brand'].head(100)
top_100_list = top_100.to_list()

#top_100_list.remove('Unknown')
top_100_list.remove('nobrand')

In [25]:
top_100_list

['OXO',
 'Amazon',
 'Nintendo',
 'Microsoft',
 'Motorola',
 'Hearst Magazines',
 'Waterpik',
 'William Shakespeare',
 'Philips Norelco',
 'Sony',
 'Presto',
 'Black Mountain',
 'Flowtron',
 'Broan',
 'GE',
 'Eureka',
 'DEWALT',
 'Koss',
 'Arctic Silver',
 'Dave Ramsey',
 'Plantronics',
 'George RR Martin',
 'Merrell',
 'JIM DUNLOP',
 'CS Lewis',
 'Mighty Bright',
 'Bergan',
 'Fiskars',
 'Celestial Seasonings',
 'Hosa',
 'Meredith Corporation',
 'Razor',
 'Range Kleen',
 'Maxell',
 'Days of Wonder',
 'CarsonDellosa',
 'Intuit',
 'Gilmour',
 'Kyser',
 'Auto Ventshade',
 'Belkin',
 'Chuckit',
 'Senseo',
 'Freud',
 'Weber',
 'Hasbro',
 'Nylabone',
 'Clover',
 'Dr Seuss',
 'Aileen Quinn',
 'Knock Knock',
 '3M',
 'CBconcept',
 'Conde Nast Publications',
 'Wellness Natural Pet Food',
 'MindWare',
 'Symantec',
 'The Elf on the Shelf',
 'CleanTools',
 'Garmin',
 'Shure',
 'James Stewart',
 'Multipet',
 'Sega',
 'MerryMakers',
 'Trusted Media Brands Inc',
 'SquareSoft',
 'Samsung',
 'Victor',
 '

In [26]:
df_tweets = pd.DataFrame(columns=['date', 'content', 'user','hashtags', 'coordinates', 'lang'])
start_date = date(2015, 12, 31)
end_date = date(2022, 8, 2)

df_collect = pd.DataFrame(itertools.islice(sntwitter.TwitterSearchScraper(
    f'{top_100[0]}  since:{start_date} until:{end_date} within: 10km' ).get_items(), 100))
            #[['date', 'content', 'user','hashtags', 'coordinates', 'lang']]

In [29]:
df_collect = pd.DataFrame(itertools.islice(sntwitter.TwitterSearchScraper(
    f'geocode:53.81653,-3.03033,320km since:2022-07-01 until:2022-07-31').get_items(), 100))

In [30]:
df_collect

,url,date,content,renderedContent,id,user,replyCount,retweetCount,likeCount,quoteCount,...,media,retweetedTweet,quotedTweet,inReplyToTweetId,inReplyToUser,mentionedUsers,coordinates,place,hashtags,cashtags
0,https://twitter.com/Imranicus/status/155353087...,2022-07-30 23:59:57+00:00,#Tories looking at the state of the economy: h...,#Tories looking at the state of the economy: h...,1553530870732132353,"{'username': 'Imranicus', 'id': 11863826114317...",0,0,2,0,...,[{'previewUrl': 'https://pbs.twimg.com/media/F...,None,None,NaN,None,None,"{'longitude': -2.659936, 'latitude': 51.399367}","{'fullName': 'Bristol, England', 'name': 'Bris...",[Tories],None
1,https://twitter.com/DavidJCourt/status/1553530...,2022-07-30 23:59:55+00:00,@TAGartwork I had a frustrating one.\n\n🟩🟩🟩⬜⬜\...,@TAGartwork I had a frustrating one.\n\n🟩🟩🟩⬜⬜\...,1553530864637808641,"{'username': 'DavidJCourt', 'id': 6642372, 'di...",1,0,0,0,...,None,None,None,1.553519e+18,"{'username': 'TAGartwork', 'id': 221710311, 'd...","[{'username': 'TAGartwork', 'id': 221710311, '...","{'longitude': -1.6128929, 'latitude': 52.3653979}","{'fullName': 'Coventry, England', 'name': 'Cov...",None,None
2,https://twitter.com/JKJeonEuphoria/status/1553...,2022-07-30 23:59:50+00:00,They're obv gonna do this right? Meaning Chica...,They're obv gonna do this right? Meaning Chica...,1553530842902822913,"{'username': 'JKJeonEuphoria', 'id': 123198962...",0,0,1,0,...,None,None,None,NaN,None,None,"{'longitude': -2.0971002, 'latitude': 52.460709}","{'fullName': 'West Bromwich, England', 'name':...",None,None
3,https://twitter.com/Mofoman360/status/15535308...,2022-07-30 23:59:48+00:00,Come on dudes who is looking forward to next w...,Come on dudes who is looking forward to next w...,1553530833381859328,"{'username': 'Mofoman360', 'id': 20820445, 'di...",1,1,5,0,...,None,None,None,NaN,None,None,"{'longitude': -1.706057, 'latitude': 53.7363694}","{'fullName': 'Leeds, England', 'name': 'Leeds'...","[mot, alaw, lufc, LeedsUnited]",None
4,https://twitter.com/sophielordx1/status/155353...,2022-07-30 23:59:47+00:00,Right. Ive bought a paddle board,Right. Ive bought a paddle board,1553530829799923712,"{'username': 'sophielordx1', 'id': 576408734, ...",0,0,0,0,...,None,None,None,NaN,None,None,"{'longitude': -2.5139084, 'latitude': 53.531967}","{'fullName': 'Bolton, England', 'name': 'Bolto...",None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,https://twitter.com/gedtraynor41/status/155352...,2022-07-30 23:55:05+00:00,@Jsize18 Give me like till tmoz I’ll show ye s...,@Jsize18 Give me like till tmoz I’ll show ye s...,1553529646590959617,"{'username': 'gedtraynor41', 'id': 10327894383...",1,0,1,0,...,None,None,None,1.553529e+18,"{'username': 'Jsize18', 'id': 1436254643714240...","[{'username': 'Jsize18', 'id': 143625464371424...","{'longitude': -2.9179613, 'latitude': 53.539352}","{'fullName': 'Ormskirk, England', 'name': 'Orm...",None,None
96,https://twitter.com/Ian_Nutten/status/15535296...,2022-07-30 23:55:05+00:00,@mhdksafa 🙏🙏,@mhdksafa 🙏🙏,1553529646339293189,"{'username': 'Ian_Nutten', 'id': 1411031857408...",0,0,0,0,...,None,None,None,1.553478e+18,"{'username': 'mhdksafa', 'id': 834078524277456...","[{'username': 'mhdksafa', 'id': 83407852427745...","{'longitude': -2.755316, 'latitude': 53.7152153}","{'fullName': 'Higher Penwortham, England', 'na...",None,None
97,https://twitter.com/___Mezzala/status/15535296...,2022-07-30 23:55:03+00:00,"Welcome back, lads ⚽️🔴 #LFC #CommunityShield h...","Welcome back, lads ⚽️🔴 #LFC #CommunityShield h...",1553529639095734272,"{'username': '___Mezzala', 'id': 1346537588987...",1,0,5,0,...,[{'thumbnailUrl': 'https://pbs.twimg.com/ext_t...,None,None,NaN,None,None,"{'longitude': -2.033651, 'latitude': 52.381063}","{'fullName': 'Birmingham, England', 'name': 'B...","[LFC, CommunityShield]",None
98,https://twitter.com/hepolytes/status/155352960...,2022-07-30 23:54:54+00:00,@Bandraidoir @GreenBagOG Too late if you're ov

In [32]:
#https://stackoverflow.com/questions/11865685/handling-a-timeout-error-in-python-sockets
import errno
import os
import signal
import functools

class TimeoutError(Exception):
    pass

def timeout(seconds=10, error_message=os.strerror(errno.ETIME)):
    def decorator(func):
        def _handle_timeout(signum, frame):
            raise TimeoutError(error_message)

        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            signal.signal(signal.SIGALRM, _handle_timeout)
            signal.alarm(seconds)
            try:
                result = func(*args, **kwargs)
            finally:
                signal.alarm(0)
            return result

        return wrapper

    return decorator

def daterange(start_date, end_date):
    for n in range(int((end_date - start_date).days)):
        yield start_date + timedelta(n)

In [33]:
'''@timeout(60)
def scrape_product(start, end, product):
    df_tweets = pd.DataFrame(columns=['date', 'content', 'user','hashtags', 'coordinates', 'lang','product'])
    df_collect = pd.DataFrame(itertools.islice(sntwitter.TwitterSearchScraper(
        f'{product} geocode:53.81653,-3.03033,320km since:{start} until:{end}').get_items(), 100000))
    df_collect['product'] = product
    df_tweets = pd.concat([df_tweets, df_collect])#, df_BBC, df_Amazon
    return df_tweets

start_date = date(2022, 5, 25)
end_date = date(2022, 8, 2)
dates = pd.date_range(start = start_date, end = end_date,freq='D').strftime('%Y-%m-%d').tolist()

count=1
df_collected = pd.DataFrame(columns=['date', 'content', 'user','hashtags', 'coordinates', 'lang','product'])
df_new = pd.DataFrame(columns=['date', 'content', 'user','hashtags', 'coordinates', 'lang','product'])
for single_date in tqdm(daterange(start_date, end_date)):
    next_date = single_date + datetime.timedelta(days=1)
    df_new = df_new.drop(df_new.index,inplace=True)
    df_collected = df_collected.drop(df_collected.index,inplace=True)
    for product in tqdm(top_100_list):
        try:
            #print(f'doing, single {single_date} next {next_date} product {product}')
            df_new = scrape_product(single_date, next_date, product)
        except TimeoutError:
            print(f'timeout, single {single_date} next {next_date} product {product}')
            continue
        df_collected = pd.concat([df_collected, df_new])
    df_collected.to_csv(f'tweets/tweets{single_date}.csv', index=False)'''




"@timeout(60)\ndef scrape_product(start, end, product):\n    df_tweets = pd.DataFrame(columns=['date', 'content', 'user','hashtags', 'coordinates', 'lang','product'])\n    df_collect = pd.DataFrame(itertools.islice(sntwitter.TwitterSearchScraper(\n        f'{product} geocode:53.81653,-3.03033,320km since:{start} until:{end}').get_items(), 100000))\n    df_collect['product'] = product\n    df_tweets = pd.concat([df_tweets, df_collect])#, df_BBC, df_Amazon\n    return df_tweets\n\nstart_date = date(2022, 5, 25)\nend_date = date(2022, 8, 2)\ndates = pd.date_range(start = start_date, end = end_date,freq='D').strftime('%Y-%m-%d').tolist()\n\ncount=1\ndf_collected = pd.DataFrame(columns=['date', 'content', 'user','hashtags', 'coordinates', 'lang','product'])\ndf_new = pd.DataFrame(columns=['date', 'content', 'user','hashtags', 'coordinates', 'lang','product'])\nfor single_date in tqdm(daterange(start_date, end_date)):\n    next_date = single_date + datetime.timedelta(days=1)\n    df_new = df

In [ ]:
@timeout(60)
def scrape_account(start, end, account):
    df_tweets = pd.DataFrame(columns=['date', 'content', 'user','hashtags', 'coordinates', 'lang','account'])
    df_collect = pd.DataFrame(itertools.islice(sntwitter.TwitterSearchScraper(
        f'from:{account} since:{start} until:{end}').get_items(), 100000)) #geocode:53.81653,-3.03033,320km
    df_collect['account'] = account
    df_tweets = pd.concat([df_tweets, df_collect])#, df_BBC, df_Amazon
    return df_tweets


start_date = date(2018, 12, 27)
end_date = date(2022, 8, 2)
dates = pd.date_range(start = start_date, end = end_date,freq='D').strftime('%Y-%m-%d').tolist()

count=1
df_collected = pd.DataFrame(columns=['date', 'content', 'user','hashtags', 'coordinates', 'lang','account'])
df_new = pd.DataFrame(columns=['date', 'content', 'user','hashtags', 'coordinates', 'lang','product'])
accounts = ['BBCBreaking', 'BBCNews', '10DowningStreet', 'RoyalFamily', 'MayorofLondon', 'Tfl',
'AmazonUK', 'British_Airways'	, 'guardiannews', 'Telegraph', 'TheSun', 'thetimes', 'DailyMirror',
'Daily_Express', 'ft', 'DailyMailUK', 'standardnews', 'EveningStandard', 'TheScotsman', 'MartinSLewis', 'MoneySavingExp', 'WhichUk',
'Burberry', 'NandosUK', 'PenguinUKBooks', 'TopShop', 'ASOS', 'StellaMcCartney', 'NETAPORTER', 'unitedbus', 'BondandMayfairX',
]
for single_date in tqdm(daterange(start_date, end_date)):
    next_date = single_date + datetime.timedelta(days=1)
    df_new = df_new.drop(df_new.index,inplace=True)
    df_collected = df_collected.drop(df_collected.index,inplace=True)
    for account in tqdm(accounts):
        try:
            #print(f'doing, single {single_date} next {next_date} product {product}')
            df_new = scrape_account(single_date, next_date, account)
        except TimeoutError:
            print(f'timeout, single {single_date} next {next_date} account {account}')
            continue
        df_collected = pd.concat([df_collected, df_new])
    df_collected.to_csv(f'tweets/tweets{single_date}-accounts.csv', index=False)


0it [00:00, ?it/s]
100%|██████████| 31/31 [01:05<00:00,  2.13s/it]
1it [01:06, 66.37s/it]
100%|██████████| 31/31 [01:12<00:00,  2.35s/it]
2it [02:19, 70.58s/it]
100%|██████████| 31/31 [01:08<00:00,  2.20s/it]
3it [03:28, 69.65s/it]
100%|██████████| 31/31 [00:58<00:00,  1.90s/it]
4it [04:27, 65.51s/it]
100%|██████████| 31/31 [01:09<00:00,  2.23s/it]
5it [05:37, 66.99s/it]
100%|██████████| 31/31 [01:01<00:00,  1.99s/it]
6it [06:39, 65.31s/it]
100%|██████████| 31/31 [01:18<00:00,  2.54s/it]
7it [07:58, 69.86s/it]
100%|██████████| 31/31 [01:25<00:00,  2.75s/it]
8it [09:24, 74.94s/it]
100%|██████████| 31/31 [01:28<00:00,  2.86s/it]
9it [10:53, 79.45s/it]
100%|██████████| 31/31 [01:04<00:00,  2.09s/it]
10it [11:58, 75.06s/it]
100%|██████████| 31/31 [01:03<00:00,  2.04s/it]
11it [13:02, 71.53s/it]
100%|██████████| 31/31 [01:24<00:00,  2.73s/it]
12it [14:27, 75.64s/it]
100%|██████████| 31/31 [01:27<00:00,  2.82s/it]
13it [15:55, 79.35s/it]
100%|██████████| 31/31 [01:32<00:00,  2.99s/it]
14it [

In [ ]:

def scrape_any(start, end):
    df_tweets = pd.DataFrame(columns=['date', 'content', 'user','hashtags', 'coordinates', 'lang','account'])
    df_collect = pd.DataFrame(itertools.islice(sntwitter.TwitterSearchScraper(
        f'geocode:53.81653,-3.03033,320km since:{start} until:{end}').get_items(), 1000)) #geocode:53.81653,-3.03033,320km
    df_collect['any'] = 'any'
    df_tweets = pd.concat([df_tweets, df_collect])#, df_BBC, df_Amazon
    return df_tweets

start_date = date(2015, 12, 31)
end_date = date(2022, 8, 2)
dates = pd.date_range(start = start_date, end = end_date,freq='D').strftime('%Y-%m-%d').tolist()

count=1
df_collected = pd.DataFrame(columns=['date', 'content', 'user','hashtags', 'coordinates', 'lang','account'])
df_new = pd.DataFrame(columns=['date', 'content', 'user','hashtags', 'coordinates', 'lang','product'])

for single_date in tqdm(daterange(start_date, end_date)):
    next_date = single_date + datetime.timedelta(days=1)
    df_new = df_new.drop(df_new.index,inplace=True)
    df_collected = df_collected.drop(df_collected.index,inplace=True)
    df_new = scrape_any(single_date, next_date)
    df_collected = pd.concat([df_collected, df_new])
    df_collected.to_csv(f'tweets/tweets{single_date}-any.csv', index=False)